# Visualization of results

Load modules and constants

In [5]:
import shutil
import os
import src.utils as utils
from src.data.dataloader import mtlDataModule

from src.data.manifests import generate_manifest

import torchvision.transforms as T
from torchvision.utils import draw_segmentation_masks
from src.visualization.draw_things import draw_bounding_boxes
from PIL import Image
from pathlib import Path

from src.utils import load_yaml
import torch

ROOT_DIR = "/zhome/3b/d/154066/repos/multitask-mayhem"

os.chdir(ROOT_DIR)

CLASS_LOOKUP = load_yaml(ROOT_DIR+"/configs/class_lookup.yaml")

Get config to initialize `pl.DataModule` and create dataloaders

In [6]:
config_file = ROOT_DIR + "/configs/debug_foo.yaml"


data_module = mtlDataModule(config_path=config_file)

data_module.config["batch_size"] = 1
data_module.config["num_workers"] = 0
data_module.config["shuffle"] = False
# print(torch.cuda.is_available())

data_module.prepare_data()
# data_module.setup(stage="fit")
# train_dataloader = data_module.train_dataloader()
# data_module.setup(stage="validate")
# valid_dataloader = data_module.val_dataloader()
data_module.setup(stage="test")
test_dataloader = data_module.test_dataloader()

100%|██████████| 5/5 [00:10<00:00,  2.14s/it]


Prepare landing folder and label names

In [9]:
sanity_check = ROOT_DIR+"/reports/figures/sanity_check"

if os.path.exists(sanity_check):
    shutil.rmtree(sanity_check)
    
os.makedirs(sanity_check, exist_ok=True)

## Sanity check labels

Loop through images and draw labels (use show if not on HPC)

In [10]:
for i, batch in enumerate(test_dataloader):
    # if i>50:
    #     break
    image, targets = batch
    

    boxes = targets[0]["boxes"]
    labels = targets[0]["labels"]
    masks = targets[0]["masks"]
    label_names = [CLASS_LOOKUP["bbox_rev"][label.item()] for label in labels]

    
    img = image[0].mul(255).type(torch.uint8)

    scores = [None]*len(boxes)


    drawn_image = draw_bounding_boxes(image=img, boxes=boxes, labels=label_names, scores=scores)
    drawn_image = draw_segmentation_masks(drawn_image, masks, alpha=0.5, colors="green")

    
    image_pil = T.ToPILImage()(drawn_image)
    image_pil.save(sanity_check+"/{}.png".format(i))
    # image_pil.show()